## Echo add primers

In [1]:
import re
import math
import numpy as np
import pandas as pd

### Source Gen

In [2]:
file = "Batch_2_primer_mix_transfer.csv"

In [3]:
df = pd.read_csv(file)

In [4]:
df

,Index,Gene
0,1,OPI1
1,2,RFX1
2,3,RGR1
3,4,RPD3
4,5,SPT3
5,6,YAP6
6,7,TFC7
7,8,OPI1
8,9,RFX1
9,10,RGR1


In [5]:
df1 = df.groupby("Gene").count().reset_index()

In [6]:
df1

,Gene,Index
0,OPI1,2
1,RFX1,2
2,RGR1,3
3,RPD3,4
4,SPT3,5
5,TFC7,7
6,YAP6,6


In [7]:
MinVol = 20
MaxVol = 60
rep = 3
max1time = 2.5

In [8]:
#genPrimerList
#how many need to be used?
df1["NeedWell"] = pd.NaT
df1["NeedVol"] =  pd.NaT
#volList = list()
for i in range(len(df1)):
    #volList.append(df1["Order"][i] * 3 * 5) #samples * 3 replicate * 5 uL primer mix
    df1.loc[i, "NeedVol"] =  df1["Index"][i] * 3 * 5
    df1.loc[i, "NeedWell"] = math.ceil(df1["NeedVol"][i]/(MaxVol - MinVol))

In [68]:
df2 = df1.sort_values(by=["Index"])
df2 = df2.rename(columns={"Index": "Num"})

In [69]:
df2

,Gene,Num,NeedWell,NeedVol
0,OPI1,2,1,30
1,RFX1,2,1,30
2,RGR1,3,2,45
3,RPD3,4,2,60
4,SPT3,5,2,75
6,YAP6,6,3,90
5,TFC7,7,3,105


In [70]:
df2.to_csv("source_vol_batch_2.csv")

In [71]:
df2["NeedWell"].sum()

14

In [72]:
def sourceGen(df):
    df1 = df.groupby("Gene").count().reset_index()
    df1["NeedWell"] = pd.NaT
    df1["NeedVol"] =  pd.NaT
    #volList = list()
    for i in range(len(df1)):
        #volList.append(df1["Order"][i] * 3 * 5) #samples * 3 replicate * 5 uL primer mix
        df1.loc[i, "NeedVol"] =  df1["Index"][i] * 3 * 5
        df1.loc[i, "NeedWell"] = math.ceil(df1["NeedVol"][i]/(MaxVol - MinVol))
    df2 = df1.sort_values(by=["Index"])
    df2 = df2.rename(columns={"Index": "Num"})
    return df2

In [73]:
df2 = sourceGen(df)

In [74]:
df2

,Gene,Num,NeedWell,NeedVol
0,OPI1,2,1,30
1,RFX1,2,1,30
2,RGR1,3,2,45
3,RPD3,4,2,60
4,SPT3,5,2,75
6,YAP6,6,3,90
5,TFC7,7,3,105


### Source Place

In [13]:
def initPlate96():
    class plate_96:
        row = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']
        col = list(range(1, 12+1))
        colNum = len(col)
        rowNum = len(row)
        df = pd.DataFrame(np.nan, index=row, columns=col)
    return plate_96

In [14]:
def initPlate384():
    class plate_384:
        row = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P']
        col = list(range(1, 24+1))
        colNum = len(col)
        rowNum = len(row)
        df = pd.DataFrame(np.nan, index=row, columns=col)
    return plate_384

In [75]:
df384 = initPlate384()

In [16]:
df384.row

['A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P']

In [17]:
df2 = df2.reset_index()

In [18]:
sourceDict = dict()
count = 0
for i in range(len(df2)):
    for j in range(df2["NeedWell"][i]):
        sourceDict[df2["Gene"][i]+"_"+str(j+1)] = str(df384.row[count]) + str(1)
        count += 1

In [19]:
sourceDict

{'OPI1_1': 'A1',
 'RFX1_1': 'B1',
 'RGR1_1': 'C1',
 'RGR1_2': 'D1',
 'RPD3_1': 'E1',
 'RPD3_2': 'F1',
 'SPT3_1': 'G1',
 'SPT3_2': 'H1',
 'YAP6_1': 'I1',
 'YAP6_2': 'J1',
 'YAP6_3': 'K1',
 'TFC7_1': 'L1',
 'TFC7_2': 'M1',
 'TFC7_3': 'N1'}

In [76]:
def sourceDictGen(source_df, plate):
    source_df = source_df.reset_index()
    sourceDict = dict()
    count = 0
    for i in range(len(source_df)):
        for j in range(source_df["NeedWell"][i]):
            sourceDict[source_df["Gene"][i]+"_"+str(j+1)] = str(plate.row[count]) + str(1)
            count += 1
    return sourceDict

In [77]:
sd = sourceDictGen(df2, df384)

In [78]:
sd

{'OPI1_1': 'A1',
 'RFX1_1': 'B1',
 'RGR1_1': 'C1',
 'RGR1_2': 'D1',
 'RPD3_1': 'E1',
 'RPD3_2': 'F1',
 'SPT3_1': 'G1',
 'SPT3_2': 'H1',
 'YAP6_1': 'I1',
 'YAP6_2': 'J1',
 'YAP6_3': 'K1',
 'TFC7_1': 'L1',
 'TFC7_2': 'M1',
 'TFC7_3': 'N1'}

In [20]:
def showSourceLoc(sourceDict, plate):
    df = plate.df
    for key in sourceDict:
        #print(sDict[key])
        index = re.findall(r'(\w+?)(\d+)', sourceDict[key])
        df.loc[index[0][0], int(index[0][1])] = key
    return df

In [21]:
sPlate = showSourceLoc(sourceDict, df384)

In [22]:
sPlate

,1,2,3,4,5,6,7,8,9,10,...,15,16,17,18,19,20,21,22,23,24
A,OPI1_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B,RFX1_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
C,RGR1_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
D,RGR1_2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
E,RPD3_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
F,RPD3_2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
G,SPT3_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
H,SPT3_2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
I,YAP6_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
J,YAP6_2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
sPlate.to_csv("source_plate_batch_2.csv")

### Source Vol Destination 

In [24]:
df96 = initPlate96()

In [25]:
destinationDict = dict()
n = 0
for num in df96.col:
    for letter in df96.row:
        destinationDict[n] = letter + str(num)
        n += 1

In [26]:
destinationDict

{0: 'A1',
 1: 'B1',
 2: 'C1',
 3: 'D1',
 4: 'E1',
 5: 'F1',
 6: 'G1',
 7: 'H1',
 8: 'A2',
 9: 'B2',
 10: 'C2',
 11: 'D2',
 12: 'E2',
 13: 'F2',
 14: 'G2',
 15: 'H2',
 16: 'A3',
 17: 'B3',
 18: 'C3',
 19: 'D3',
 20: 'E3',
 21: 'F3',
 22: 'G3',
 23: 'H3',
 24: 'A4',
 25: 'B4',
 26: 'C4',
 27: 'D4',
 28: 'E4',
 29: 'F4',
 30: 'G4',
 31: 'H4',
 32: 'A5',
 33: 'B5',
 34: 'C5',
 35: 'D5',
 36: 'E5',
 37: 'F5',
 38: 'G5',
 39: 'H5',
 40: 'A6',
 41: 'B6',
 42: 'C6',
 43: 'D6',
 44: 'E6',
 45: 'F6',
 46: 'G6',
 47: 'H6',
 48: 'A7',
 49: 'B7',
 50: 'C7',
 51: 'D7',
 52: 'E7',
 53: 'F7',
 54: 'G7',
 55: 'H7',
 56: 'A8',
 57: 'B8',
 58: 'C8',
 59: 'D8',
 60: 'E8',
 61: 'F8',
 62: 'G8',
 63: 'H8',
 64: 'A9',
 65: 'B9',
 66: 'C9',
 67: 'D9',
 68: 'E9',
 69: 'F9',
 70: 'G9',
 71: 'H9',
 72: 'A10',
 73: 'B10',
 74: 'C10',
 75: 'D10',
 76: 'E10',
 77: 'F10',
 78: 'G10',
 79: 'H10',
 80: 'A11',
 81: 'B11',
 82: 'C11',
 83: 'D11',
 84: 'E11',
 85: 'F11',
 86: 'G11',
 87: 'H11',
 88: 'A12',
 89: 'B12',
 9

In [79]:
def destinationDictGen(plate):
    destinationDict = dict()
    n = 0
    for num in plate.col:
        for letter in plate.row:
            destinationDict[n] = letter + str(num)
            n += 1
    return destinationDict

In [81]:
def s2d(sourceDict, destinationDict, df):
    df3 = pd.DataFrame()
    columns = ["Index", "Gene", "Source", "Destination", "Vol", "Time"]
    df3 = pd.DataFrame(np.nan, index=range(0, len(df) * 3 *2), columns=columns)
    source_df = pd.DataFrame(sourceDict.items(), columns=['ID', 'Loc'])
    source_df["Vol"] = 40
    s = -1
    d = -1
    num = 1
    vol = 0
    for i in range(len(df3)):
        #how many vol used? count for every source well
        if i % 2 == 0:
            d += 1
        if i % 6 == 0:
            s += 1
        df3.loc[i, "Index"] = str(df["Index"][s])
        df3.loc[i, "Gene"] = str(df["Gene"][s])
        #print(i)
        df3.loc[i, "Time"] = 1
        df3.loc[i, "Source"] = sourceDict[df3.loc[i, "Gene"] + "_" + str(math.ceil(df3.loc[df3["Gene"] == df["Gene"][s]]["Time"].sum() / 16))] # num should changed
        df3.loc[i, "Destination"] = destinationDict[d]
        df3.loc[i, "Vol"] = 2500
    return df3

In [82]:
s2d_1 = s2d(sourceDict, destinationDict, df)

In [83]:
s2d_1

,Index,Gene,Source,Destination,Vol,Time
0,1,OPI1,A1,A1,2500.0,1.0
1,1,OPI1,A1,A1,2500.0,1.0
2,1,OPI1,A1,B1,2500.0,1.0
3,1,OPI1,A1,B1,2500.0,1.0
4,1,OPI1,A1,C1,2500.0,1.0
...,...,...,...,...,...,...
169,29,TFC7,N1,E11,2500.0,1.0
170,29,TFC7,N1,F11,2500.0,1.0
171,29,TFC7,N1,F11,2500.0,1.0
172,29,TFC7,N1,G11,2500.0,1.0


In [38]:
df3 = pd.DataFrame()

In [39]:
columns = ["Index", "Gene", "Source", "Destination", "Vol", "Time"]

In [40]:
df3 = pd.DataFrame(np.nan, index=range(0, len(df) * 3 *2), columns=columns)

In [41]:
df3

,Index,Gene,Source,Destination,Vol,Time
0,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
169,NaN,NaN,NaN,NaN,NaN,NaN
170,NaN,NaN,NaN,NaN,NaN,NaN
171,NaN,NaN,NaN,NaN,NaN,NaN
172,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
sourceDict

{'OPI1_1': 'A1',
 'RFX1_1': 'B1',
 'RGR1_1': 'C1',
 'RGR1_2': 'D1',
 'RPD3_1': 'E1',
 'RPD3_2': 'F1',
 'SPT3_1': 'G1',
 'SPT3_2': 'H1',
 'YAP6_1': 'I1',
 'YAP6_2': 'J1',
 'YAP6_3': 'K1',
 'TFC7_1': 'L1',
 'TFC7_2': 'M1',
 'TFC7_3': 'N1'}

In [43]:
source_df = pd.DataFrame(sourceDict.items(), columns=['ID', 'Loc'])

In [44]:
source_df["Vol"] = 40

In [80]:
source_df

,ID,Loc,Vol
0,OPI1_1,A1,40
1,RFX1_1,B1,40
2,RGR1_1,C1,40
3,RGR1_2,D1,40
4,RPD3_1,E1,40
5,RPD3_2,F1,40
6,SPT3_1,G1,40
7,SPT3_2,H1,40
8,YAP6_1,I1,40
9,YAP6_2,J1,40


In [49]:
s = -1
d = -1
num = 1
vol = 0
for i in range(len(df3)):
    #how many vol used? count for every source well
    if i % 2 == 0:
        d += 1
    if i % 6 == 0:
        s += 1
    df3.loc[i, "Index"] = str(df["Index"][s])
    df3.loc[i, "Gene"] = str(df["Gene"][s])
    #print(i)
    df3.loc[i, "Time"] = 1
    df3.loc[i, "Source"] = sourceDict[df3.loc[i, "Gene"] + "_" + str(math.ceil(df3.loc[df3["Gene"] == df["Gene"][s]]["Time"].sum() / 16))] # num should changed
    df3.loc[i, "Destination"] = destinationDict[d]
    df3.loc[i, "Vol"] = 2500
    
    #source_df.loc[source_df["ID"] == df3.loc[i, "Gene"] + "_" + str(num)].reset_index()["Vol"][0] -= 2.5
    #vol += 2.5 #uL per time #for each sample!!!
    #set for each source!
    #if source_df.loc[source_df["ID"] == df3.loc[i, "Gene"] + "_" + str(num)].reset_index()["Vol"][0] <= 0:
        #num += 1 #this num should be source specific.
        #vol = 0

In [50]:
df3

,Index,Gene,Source,Destination,Vol,Time
0,1,OPI1,A1,A1,2500.0,1.0
1,1,OPI1,A1,A1,2500.0,1.0
2,1,OPI1,A1,B1,2500.0,1.0
3,1,OPI1,A1,B1,2500.0,1.0
4,1,OPI1,A1,C1,2500.0,1.0
...,...,...,...,...,...,...
169,29,TFC7,N1,E11,2500.0,1.0
170,29,TFC7,N1,F11,2500.0,1.0
171,29,TFC7,N1,F11,2500.0,1.0
172,29,TFC7,N1,G11,2500.0,1.0


In [84]:
def desOut(df3):
    col = ["Source Well", "Transfer Volume", "Destination Well", "ID", "Gene"]
    df4 = pd.DataFrame(columns=col)
    df4.loc[:, "Source Well"] = df3["Source"]
    df4.loc[:, "Destination Well"] = df3["Destination"]
    df4.loc[:, "Transfer Volume"] = df3["Vol"]
    df4.loc[:, "ID"] = df3["Index"]
    df4.loc[:, "Gene"] = df3["Gene"]
    return df4

In [51]:
col = ["Source Well", "Transfer Volume", "Destination Well", "ID", "Gene"]

In [52]:
df4 = pd.DataFrame(columns=col)

In [54]:
df4.loc[:, "Source Well"] = df3["Source"]
df4.loc[:, "Destination Well"] = df3["Destination"]
df4.loc[:, "Transfer Volume"] = df3["Vol"]
df4.loc[:, "ID"] = df3["Index"]
df4.loc[:, "Gene"] = df3["Gene"]

In [55]:
df4

,Source Well,Transfer Volume,Destination Well,ID,Gene
0,A1,2500.0,A1,1,OPI1
1,A1,2500.0,A1,1,OPI1
2,A1,2500.0,B1,1,OPI1
3,A1,2500.0,B1,1,OPI1
4,A1,2500.0,C1,1,OPI1
...,...,...,...,...,...
169,N1,2500.0,E11,29,TFC7
170,N1,2500.0,F11,29,TFC7
171,N1,2500.0,F11,29,TFC7
172,N1,2500.0,G11,29,TFC7


In [56]:
df4.to_csv("source2destination_transfer_batch_2.csv")

In [57]:
s2d = pd.Series(df4["Source Well"].values,index=df4["Destination Well"]).to_dict()

In [59]:
len(s2d)

87

In [60]:
def showLoc(sourceDict, plate):
    df = plate.df
    for key in sourceDict:
        #print(sDict[key])
        index = re.findall(r'(\w+?)(\d+)', key)
        df.loc[index[0][0], int(index[0][1])] = sourceDict[key]
    return df

In [61]:
s2dPlate = showLoc(s2d, df96)

In [62]:
s2dPlate

,1,2,3,4,5,6,7,8,9,10,11,12
A,A1,C1,I1,B1,E1,L1,G1,M1,J1,J1,K1,NaN
B,A1,E1,I1,B1,G1,L1,G1,F1,J1,J1,M1,NaN
C,A1,E1,L1,B1,G1,C1,H1,F1,M1,J1,N1,NaN
D,B1,E1,L1,C1,G1,C1,I1,F1,M1,M1,N1,NaN
E,B1,G1,L1,C1,I1,D1,I1,H1,M1,M1,N1,NaN
F,B1,G1,A1,C1,I1,E1,J1,H1,H1,M1,N1,NaN
G,C1,G1,A1,E1,I1,E1,L1,H1,H1,J1,N1,NaN
H,C1,I1,A1,E1,L1,F1,L1,J1,H1,K1,NaN,NaN


In [63]:
s2dPlate.to_csv("source2destination_plate_batch_2.csv")

In [ ]:
python echo_primer_transfer.py -i /home/chenjunyu/UIUC/Lab/Echo/Batch_2_primer_mix_transfer.csv -o /home/chenjunyu/UIUC/Lab/Echo -b batch_2

In [ ]:
python echo_primer_transfer.py -i /home/chenjunyu/UIUC/Lab/Echo/Batch_3_primer_mix_transfer.csv -o /home/chenjunyu/UIUC/Lab/Echo -b batch_3

In [ ]:
python echo_primer_transfer.py -i /home/chenjunyu/UIUC/Lab/Echo/Batch_3_primer_mix_transfer_rep_1.csv -o /home/chenjunyu/UIUC/Lab/Echo -b Batch_3_rep_1

In [ ]:
python echo_primer_transfer.py -i /home/chenjunyu/UIUC/Lab/Echo/Batch_4_primer_mix_transfer.csv -o /home/chenjunyu/UIUC/Lab/Echo -b Batch_4

In [ ]:
python echo_primer_transfer.py -i /home/chenjunyu/UIUC/Lab/Echo/Batch_5_primer_mix_transfer.csv -o /home/chenjunyu/UIUC/Lab/Echo -b Batch_5